In [453]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import json

In [455]:
api_key=input('Please enter your API: ')

In [456]:
#A function that captures city states using the Google Geolocation service.
def countrys (address):
    try:
        response = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address, api_key))
        response=response.json()
        country = response['results'][0]['formatted_address'].split(', ')[-1]
        return country
    except:
        return None   

In [457]:
data=pd.read_csv("destinations_LP_crawler_Ex5.csv",encoding='UTF-8')

In [458]:
#Establishes a state for each city
data['country'] = [countrys(x) for x in data['city']]
data

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
...,...,...,...
72,Venice,Venice,Italy
73,Vienna,Vienna,Austria
74,Warsaw,Warsaw,Poland
75,Zakynthos,Zakynthos,Greece


In [459]:
#Check if there are cities that have failed to absorb a state
data.loc[data['country'].isnull()]

,city,city_LP,country


In [460]:
#When there is a problem identifying the country, use the city_LP column
data['city']=np.where(data['country'].isnull(), data['city_LP'].str.strip("/")[1],data['city'])
data['country']=np.where(data['country'].isnull(), countrys(data['city']),data['country'])
data

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain
...,...,...,...
72,Venice,Venice,Italy
73,Vienna,Vienna,Austria
74,Warsaw,Warsaw,Poland
75,Zakynthos,Zakynthos,Greece


In [461]:
#A test that no city remains without a state
data.loc[data['country'].isnull()]

,city,city_LP,country


In [462]:
data['combination']= data['country']+'/'+data['city_LP']
data

,city,city_LP,country,combination
0,Antalya,mediterranean-coast/antalya,Turkey,Turkey/mediterranean-coast/antalya
1,Athens,Athens,Greece,Greece/Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus,Cyprus/the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok,Thailand,Thailand/Bangkok
4,Barcelona,Barcelona,Spain,Spain/Barcelona
...,...,...,...,...
72,Venice,Venice,Italy,Italy/Venice
73,Vienna,Vienna,Austria,Austria/Vienna
74,Warsaw,Warsaw,Poland,Poland/Warsaw
75,Zakynthos,Zakynthos,Greece,Greece/Zakynthos


In [463]:
def get_recipe_info(recipe):
    url = "https://www.lonelyplanet.com"
    link=str(url)+"/"+str(recipe)
    recipe_dict = dict()
    import requests
    from bs4 import BeautifulSoup
    try:
        response = requests.get(link)
        if not response.status_code == 200:
            return recipe_dict
        result_page = BeautifulSoup(response.content,'html.parser')
        description_list = ""
        for description in result_page.find_all('div',class_='readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg'):
            description_list += description.get_text()
        return description_list
    except:
        return description_list

In [464]:
data['description'] = [get_recipe_info(x) for x in data['combination']]
data

,city,city_LP,country,combination,description
0,Antalya,mediterranean-coast/antalya,Turkey,Turkey/mediterranean-coast/antalya,Once seen simply as the gateway to the Turkish...
1,Athens,Athens,Greece,Greece/Athens,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus,Cyprus/the-republic-of-cyprus/Agia-napa,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Bangkok,Thailand,Thailand/Bangkok,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Barcelona,Spain,Spain/Barcelona,Barcelona is an enchanting seaside city with b...
...,...,...,...,...,...
72,Venice,Venice,Italy,Italy/Venice,Imagine the audacity of building a city of mar...
73,Vienna,Vienna,Austria,Austria/Vienna,Baroque streetscapes and imperial palaces set ...
74,Warsaw,Warsaw,Poland,Poland/Warsaw,"A phoenix arisen from the ashes, Poland's capi..."
75,Zakynthos,Zakynthos,Greece,Greece/Zakynthos,"Zakynthos, also known by its Italian name Zant..."


In [465]:
#Delete unnecessary columns
data=data.drop(['city_LP', 'combination'], axis=1)
data

,city,country,description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
72,Venice,Italy,Imagine the audacity of building a city of mar...
73,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
74,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
75,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."


In [466]:
#Upload the data to an Excel file
data.to_csv('LP_destinations.csv',index=False)

In [467]:
data.set_index('city',inplace=True)
data

,country,description
city,,
Antalya,Turkey,Once seen simply as the gateway to the Turkish...
Athens,Greece,"With equal measures of grunge and grace, Athen..."
Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...
Venice,Italy,Imagine the audacity of building a city of mar...
Vienna,Austria,Baroque streetscapes and imperial palaces set ...
Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
